### 1 - Import and clean test data

In [129]:
import pandas as pd
import pandas_ta as ta
from polygon import RESTClient
import plotly.graph_objects as go
from plotly.offline import plot

client = RESTClient(api_key="n__Yap_rQ8p5ZSPCBnjxBv7zwGymmKHZ")

# Define the ticker symbol and date range

dataRequest = client.get_aggs(ticker = "AAPL",
                              multiplier = 5,
                              timespan = "minute",
                              from_ = "2025-01-01",
                              to = "2100-01-01",
                              limit=50000)

df = pd.DataFrame(dataRequest)

# Convert timestamp to datetime, set as index, and name it "Date"
df = df.set_index(pd.to_datetime(df['timestamp'], unit='ms')).rename_axis("Date")

# Drop the timestamp column as it's now redundant
df = df.drop('timestamp', axis=1)


#df['Date'] = pd.to_datetime(df['timestamp'], unit='ms')
#df.set_index(df['Date'], inplace=True)

# Rename columns to have capitalized first letters
df = df.rename(columns={
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close'
})

df

#generate plotly
#fig = go.Figure(data=[go.Candlestick(x=df.index,
                                     #open=df['Open'],
                                     #high=df['High'],
                                     #low=df['Low'],
                                     #close=df['Close'])])

#plot(fig, auto_open=True)





,Open,High,Low,Close,volume,vwap,transactions,otc
Date,,,,,,,,
2025-01-02 09:00:00,251.36,251.42,250.6000,251.1300,5181.0,251.2158,333,None
2025-01-02 09:05:00,251.15,251.45,251.1500,251.3900,5247.0,251.3060,138,None
2025-01-02 09:10:00,251.54,251.54,251.4200,251.4200,1331.0,251.4770,62,None
2025-01-02 09:15:00,251.41,251.70,251.3900,251.6600,6557.0,251.6117,194,None
2025-01-02 09:20:00,251.66,251.90,251.6600,251.9000,4873.0,251.8040,107,None
...,...,...,...,...,...,...,...,...
2025-03-01 00:35:00,241.86,241.90,241.6855,241.9000,1347.0,241.7712,19,None
2025-03-01 00:40:00,241.85,241.94,241.8500,241.9000,2193.0,241.8882,29,None
2025-03-01 00:45:00,241.90,241.94,241.9000,241.9400,1939.0,241.9066,28,None


In [130]:
def populate_indicators(df):
    df["EMA_slow"]=ta.ema(df.Close, length=50)
    df["EMA_fast"]=ta.ema(df.Close, length=30)
    df['RSI']=ta.rsi(df.Close, length=10)
    df['ATR']=ta.atr(df.High, df.Low, df.Close, length=7)
    
    my_bbands = ta.bbands(df.Close, length=15, std=1.5)
    df=df.join(my_bbands)

    return df

df = populate_indicators(df)
df

,Open,High,Low,Close,volume,vwap,transactions,otc,EMA_slow,EMA_fast,RSI,ATR,BBL_15_1.5,BBM_15_1.5,BBU_15_1.5,BBB_15_1.5,BBP_15_1.5
Date,,,,,,,,,,,,,,,,,
2025-01-02 09:00:00,251.36,251.42,250.6000,251.1300,5181.0,251.2158,333,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-02 09:05:00,251.15,251.45,251.1500,251.3900,5247.0,251.3060,138,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-02 09:10:00,251.54,251.54,251.4200,251.4200,1331.0,251.4770,62,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-02 09:15:00,251.41,251.70,251.3900,251.6600,6557.0,251.6117,194,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-02 09:20:00,251.66,251.90,251.6600,251.9000,4873.0,251.8040,107,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-01 00:35:00,241.86,241.90,241.6855,241.9000,1347.0,241.7712,19,None,241.215738,241.713852,49.563306,0.153100,241.890028,241.992900,242.095772,0.085021,0.048468
2025-03-01 00:40:00,241.85,241.94,241.8500,241.9000,2193.0,241.8882,29,None,241.242572,241.725862,49.563306,0.144086,241.877611,241.982233,242.086856,0.086471,0.106999
2025-03-01 00:45:00,241.90,241.94,241.9000,241.9400,1939.0,241.9066,28,None,241.269922,241.739677,52.313440,0.129216,241.872327,241.976900,242.081473,0.086432,0.323568


In [131]:
def ema_signal(df, current_candle, backcandles):
    df_slice = df.reset_index().copy()
    # Get the range of candles to consider
    start = max(0, current_candle - backcandles)
    end = current_candle
    relevant_rows = df_slice.iloc[start:end]

    # Check if all EMA_fast values are below EMA_slow values
    if all(relevant_rows["EMA_fast"] < relevant_rows["EMA_slow"]):
        return 1
    elif all(relevant_rows["EMA_fast"] > relevant_rows["EMA_slow"]):
        return 2
    else:
        return 0

df=df[-10000:-1]
from tqdm import tqdm
tqdm.pandas()
df.reset_index(inplace=True)
df['EMASignal'] = df.progress_apply(lambda row: ema_signal(df, row.name, 7) , axis=1) #if row.name >= 20 else 0

100%|██████████| 7290/7290 [00:06<00:00, 1181.82it/s]
/tmp/ipykernel_2823533/721800138.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [132]:
df

,Date,Open,High,Low,Close,volume,vwap,transactions,otc,EMA_slow,EMA_fast,RSI,ATR,BBL_15_1.5,BBM_15_1.5,BBU_15_1.5,BBB_15_1.5,BBP_15_1.5,EMASignal
0,2025-01-02 09:00:00,251.36,251.42,250.6000,251.13,5181.0,251.2158,333,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,2025-01-02 09:05:00,251.15,251.45,251.1500,251.39,5247.0,251.3060,138,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,2025-01-02 09:10:00,251.54,251.54,251.4200,251.42,1331.0,251.4770,62,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,2025-01-02 09:15:00,251.41,251.70,251.3900,251.66,6557.0,251.6117,194,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,2025-01-02 09:20:00,251.66,251.90,251.6600,251.90,4873.0,251.8040,107,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7285,2025-03-01 00:30:00,241.61,241.92,241.6100,241.92,438.0,241.7363,14,None,241.187809,241.701014,50.748630,0.139533,241.905040,242.003567,242.102093,0.081426,0.075917,2
7286,2025-03-01 00:35:00,241.86,241.90,241.6855,241.90,1347.0,241.7712,19,None,241.215738,241.713852,49.563306,0.153100,241.890028,241.992900,242.095772,0.085021,0.048468,2
7287,2025-03-01 00:40:00,241.85,241.94,241.8500,241.90,2193.0,241.8882,29,None,241.242572,241.725862,49.563306,0.144086,241.877611,241.982233,242.086856,0.086471,0.106999,2
7288,2025-03-01 00:45:00,241.90,241.94,241.9000,241.94,1939.0,241.9066,28,None,241.269922,241.739677,52.313440,0.129216,241.872327,241.976900,242.081473,0.086432,0.323568,2


In [133]:
def total_signal(df, current_candle, backcandles):
    #Uptrend
    if (ema_signal(df, current_candle, backcandles)==2
        and df.Close[current_candle]<=df['BBL_15_1.5'][current_candle]
        #and df.RSI[current_candle]<60
        ):
            return 2
    #Downtrend
    if (ema_signal(df, current_candle, backcandles)==1
        and df.Close[current_candle]>=df['BBU_15_1.5'][current_candle]
        #and df.RSI[current_candle]>40
        ):
    
            return 1
    return 0
        
df['TotalSignal'] = df.progress_apply(lambda row: total_signal(df, row.name, 7), axis=1)

100%|██████████| 7290/7290 [00:13<00:00, 535.30it/s]
/tmp/ipykernel_2823533/1208225980.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [134]:
df[df.TotalSignal != 0].head(30)

,Date,Open,High,Low,Close,volume,vwap,transactions,otc,EMA_slow,EMA_fast,RSI,ATR,BBL_15_1.5,BBM_15_1.5,BBU_15_1.5,BBB_15_1.5,BBP_15_1.5,EMASignal,TotalSignal
97,2025-01-02 17:10:00,244.6800,245.2500,244.6800,245.1137,291800.0,245.0061,4718,None,246.551099,245.637439,49.626296,0.491919,244.450195,244.769933,245.089671,0.261256,1.037576,1,1
132,2025-01-02 20:05:00,242.4800,242.8300,242.4200,242.8100,373371.0,242.6043,5680,None,243.514098,242.782108,57.471529,0.416671,242.112151,242.387980,242.663809,0.227593,1.265003,1,1
133,2025-01-02 20:10:00,242.8050,243.4400,242.7100,243.2500,604494.0,243.1036,8076,None,243.503741,242.812294,65.093603,0.461432,242.039572,242.455980,242.872388,0.343492,1.453415,1,1
134,2025-01-02 20:15:00,243.2300,243.4300,243.0900,243.3800,484444.0,243.2910,6545,None,243.498888,242.848920,67.033229,0.444085,241.991755,242.527313,243.062871,0.441647,1.296073,1,1
137,2025-01-02 20:30:00,243.2500,243.5900,243.0700,243.3450,526159.0,243.3724,7918,None,243.466514,242.915344,63.126150,0.426012,242.073820,242.708387,243.342953,0.522904,1.001613,1,1
182,2025-01-03 00:25:00,243.3300,243.5000,243.3300,243.4200,33827.0,243.3390,213,None,243.428653,243.360025,57.235916,0.101756,243.172039,243.276267,243.380494,0.085687,1.189517,1,1
183,2025-01-03 00:30:00,243.4600,243.5000,243.4500,243.4600,1635.0,243.4752,61,None,243.429882,243.366475,60.196393,0.098648,243.172930,243.293320,243.413710,0.098967,1.192251,1,1
184,2025-01-03 00:35:00,243.6000,243.9400,243.4500,243.8600,42827.0,243.7073,273,None,243.446749,243.398315,77.501935,0.154555,243.085107,243.329320,243.573533,0.200726,1.586512,1,1
185,2025-01-03 00:40:00,243.9300,243.9500,243.7200,243.8200,6601.0,243.8464,189,None,243.461387,243.425521,73.930491,0.165333,243.059156,243.363987,243.668818,0.250514,1.247977,1,1
186,2025-01-03 00:45:00,243.8804,243.9000,243.8800,243.8800,858.0,243.8882,26,None,243.477803,243.454842,75.789907,0.153143,243.051537,243.406893,243.762250,0.291986,1.165678,1,1


In [135]:
import numpy as np
def pointpos(x):
    if x['TotalSignal']==2:
        return x['Low']-1e-3
    elif x['TotalSignal']==1:
        return x['High']+1e-3
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

/tmp/ipykernel_2823533/2345118794.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [136]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
st=100
dfpl = df[st:st+350]
#dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),

                go.Scatter(x=dfpl.index, y=dfpl['BBL_15_1.5'], 
                           line=dict(color='green', width=1), 
                           name="BBL"),
                go.Scatter(x=dfpl.index, y=dfpl['BBU_15_1.5'], 
                           line=dict(color='green', width=1), 
                           name="BBU"),
                go.Scatter(x=dfpl.index, y=dfpl['EMA_fast'], 
                           line=dict(color='black', width=1), 
                           name="EMA_fast"),
                go.Scatter(x=dfpl.index, y=dfpl['EMA_slow'], 
                           line=dict(color='blue', width=1), 
                           name="EMA_slow")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="entry")

fig.show()

In [137]:
def SIGNAL():
    return df.TotalSignal

In [138]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    mysize = 3000
    slcoef = 1.1
    TPSLRatio = 1.5
    rsi_length = 16
    
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
        #df['RSI']=ta.rsi(df.Close, length=self.rsi_length)

    def next(self):
        super().next()
        slatr = self.slcoef*self.data.ATR[-1]
        TPSLRatio = self.TPSLRatio

        # if len(self.trades)>0:
        #     if self.trades[-1].is_long and self.data.RSI[-1]>=90:
        #         self.trades[-1].close()
        #     elif self.trades[-1].is_short and self.data.RSI[-1]<=10:
        #         self.trades[-1].close()
        
        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(df, MyStrat, cash=250, margin=1/30)

/tmp/ipykernel_2823533/2590162779.py:36: UserWarning:

Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).

/tmp/ipykernel_2823533/2590162779.py:36: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



In [139]:
bt.run()

Start                                     0.0
End                                    7289.0
Duration                               7289.0
Exposure Time [%]                         0.0
Equity Final [$]                        250.0
Equity Peak [$]                         250.0
Return [%]                                0.0
Buy & Hold Return [%]                -3.67539
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                  0.0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

In [140]:
bt.plot()

GridPlot(id='p1140', ...)